In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import tensorflow as tf

In [2]:
import time

In [3]:
# 파일 경로 설정
file_path = ".\통합데이터_전처리_당뇨고혈압추가.csv"
health_data = pd.read_csv(file_path)

C:\Users\Master\AppData\Local\Temp\ipykernel_13420\113614494.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  health_data = pd.read_csv(file_path)


In [4]:
# 데이터 준비
X = health_data[['성별', '연령대코드', '신장', '체중', '수축기 혈압', '이완기 혈압']]
y = health_data['이상지질혈증']

# 범주형 변수 처리 (성별, 연령대코드)
X = pd.get_dummies(X, columns=['성별', '연령대코드'], drop_first=True)

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# y를 숫자 레이블로 변환
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

# LSTM 입력 형태로 변환 (samples, timesteps, features)
X_train_lstm = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (샘플 수, 특성 수, 채널 수)
X_test_lstm = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 구축
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

c:\Users\Master\anaconda3\envs\uni\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# 모델 학습
history = model.fit(X_train_lstm, y_train, 
                    validation_data=(X_test_lstm, y_test), 
                    epochs=4, 
                    batch_size=32)

# 예측 및 평가
y_pred = (model.predict(X_test_lstm) > 0.4).astype("int32")
print("Classification Report:\n", classification_report(y_test, y_pred))

y_pred_probabilities = model.predict(X_test_lstm)
print("예측 확률 값:\n", y_pred_probabilities)

# 전체 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Overall Accuracy: {accuracy:.4f}")

Epoch 1/4
65765/65765 ━━━━━━━━━━━━━━━━━━━━ 444s 7ms/step - accuracy: 0.6348 - loss: 0.6433 - val_accuracy: 0.6402 - val_loss: 0.6370
Epoch 2/4
65765/65765 ━━━━━━━━━━━━━━━━━━━━ 423s 6ms/step - accuracy: 0.6430 - loss: 0.6355 - val_accuracy: 0.6424 - val_loss: 0.6354
Epoch 3/4
65765/65765 ━━━━━━━━━━━━━━━━━━━━ 502s 8ms/step - accuracy: 0.6426 - loss: 0.6356 - val_accuracy: 0.6428 - val_loss: 0.6352
Epoch 4/4
65765/65765 ━━━━━━━━━━━━━━━━━━━━ 515s 8ms/step - accuracy: 0.6431 - loss: 0.6353 - val_accuracy: 0.6429 - val_loss: 0.6351
16442/16442 ━━━━━━━━━━━━━━━━━━━━ 54s 3ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.14      0.23    206903
           1       0.63      0.96      0.76    319215

    accuracy                           0.64    526118
   macro avg       0.65      0.55      0.50    526118
weighted avg       0.65      0.64      0.55    526118

16442/16442 ━━━━━━━━━━━━━━━━━━━━ 49s 3ms/step
예측 확률 값:
 [[0.80809236]
 

In [ ]:
model.save("./dyslipidemia_model.h5")
print("모델이 .h5 형식으로 저장되었습니다.")

모델이 .h5 형식으로 저장되었습니다.


In [8]:
from tensorflow.keras.models import load_model

# 모델 로드
loaded_model = load_model("dyslipidemia_model.h5")
print("저장된 모델이 성공적으로 로드되었습니다.")

# 로드된 모델로 예측
y_pred_loaded = (loaded_model.predict(X_test_lstm) > 0.4).astype("int32")
print("로드된 모델의 Classification Report:\n", classification_report(y_test, y_pred_loaded))

# 로드된 모델의 정확도 확인
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print(f"로드된 모델의 Overall Accuracy: {accuracy_loaded:.4f}")

저장된 모델이 성공적으로 로드되었습니다.
16442/16442 ━━━━━━━━━━━━━━━━━━━━ 54s 3ms/step
로드된 모델의 Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.14      0.23    206903
           1       0.63      0.96      0.76    319215

    accuracy                           0.64    526118
   macro avg       0.65      0.55      0.50    526118
weighted avg       0.65      0.64      0.55    526118

로드된 모델의 Overall Accuracy: 0.6357


### 예시 테스트 + 추론 시간

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import time

# 사용자로부터 입력된 데이터 (리스트 형태 제거)
input_data = {
    '성별': 1,  # 예: 1은 남성, 0은 여성
    '연령대코드': 2,  # 예: 2는 30대, 1은 20대 등
    '신장': 200.0,  # 예: 170 cm
    '체중': 200.0,  # 예: 65 kg
    '수축기 혈압': 120,  # 예: 120 mmHg
    '이완기 혈압': 80  # 예: 80 mmHg
}

# 입력 데이터를 DataFrame으로 변환 (리스트가 아닌 단일 값 처리)
input_df = pd.DataFrame([input_data])

# 범주형 변수 처리 (pd.get_dummies 사용)
input_df_processed = pd.get_dummies(input_df, drop_first=True)

# 스케일링: 모델에 사용된 X_train과 동일한 방식으로 스케일링
scaler = StandardScaler()
input_scaled = scaler.fit_transform(input_df_processed)

# 첫 번째 테스트 데이터에 대해 추론 시간 측정
start_time = time.time()  # 시작 시간 기록

# XGBoost 모델 예측
prediction = xgb_model.predict(input_scaled)

end_time = time.time()  # 끝 시간 기록

# 추론 시간 계산
inference_time = end_time - start_time
print(f"데이터 하나에 대한 추론 시간: {inference_time}초")

# 예측 결과 출력
print("입력된 데이터:", input_data)
print("예측 결과:", prediction)


1/1 [==============================] - 0s 14ms/step
입력된 데이터: {'성별': [1], '연령대코드': [2], '신장': [150], '체중': [80], '수축기 혈압': [120], '이완기 혈압': [80]}
예측 결과: [[1]]
데이터 하나에 대한 추론 시간: 0.04375433921813965초
